# This is standalone notebook intend for running locally or on a server

In [ ]:
!pip install ultralytics
!pip install roboflow
!pip install python-dotenv

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("API_KEY")

## Get Dataset

In [ ]:
from roboflow import Roboflow

rf = Roboflow(api_key=api_key)
project = rf.workspace("roboflow-jvuqo").project("football-players-detection-3zvbc")
version = project.version(14)
dataset = version.download("yolov11")

# Training

In [ ]:
import torch

torch.cuda.empty_cache()

# Enable expandable segments (run before initializing tensors)
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# Monitor memory usage
print(torch.cuda.memory_summary())

In [ ]:
from pathlib import Path

model_name = 'best.pt' # The starting model weights
num_epochs = 50
image_size = 640
batch_size = 6
data_yaml_path = Path(dataset.location) / 'data.yaml'

In [ ]:
from ultralytics import YOLO

model = YOLO(model_name)

print(f"Starting training with model: {model_name}")
print(f"Dataset config: {data_yaml_path}")
print(f"Epochs: {num_epochs}, Image Size: {image_size}, Batch Size: {batch_size}")

results = model.train(
    data=data_yaml_path,
    epochs=num_epochs,
    imgsz=image_size,
    batch=batch_size,
    amp=True
)